HELP

In [ ]:
def show_commands():
    """
    Shows the available commands.
    """
    print("I comandi disponibili sono i seguenti:\n"
          "\taggiungi: aggiungi un prodotto al magazzino\n"
          "\telenca: elenca i prodotti in magazzino\n"
          "\tvendita: registra una vendita effettuata\n"
          "\tprofitti: mostra i profitti totali\n"
          "\taiuto: mostra i possibili comandi\n"
          "\tchiudi: esci dal programma")

 ADD PRODUCTS

In [ ]:
def add_to_warehouse():
    """
    Adds a new product to the warehouse, if it is already present updates the amount.
    """
    item_name = input("Nome del prodotto: ")
    if check_item_presence(item_name) == True:
        update_amount(item_name)
    else:
        add_new_item(item_name)

In [ ]:
def check_item_presence(item_name):
    """
    Check if the product is already in stock.
    
    Arg:
    item_name (str): Product's name.
    
    Return:
    True (bool): The product is present.
    False (bool): The product isn't present.
    """
    with open("warehouse.json") as json_file:
        data = json.load(json_file)
    if item_name in data:
        return True
    return False

In [ ]:
def update_amount(item_name):
    """
    Updates the amount of a product already in stock and prints the operation performed on the screen.
    
    Arg:
    item_name (str): Product's name.
    """
    print("Quantità:")
    amount_to_add = is_float()
    with open("warehouse.json") as json_file:
        data = json.load(json_file)
    prev_amount = data[item_name]["amount"]
    data[item_name]["amount"] = prev_amount + amount_to_add
    with open("warehouse.json", "w") as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)
    print(f"AGGIUNTO:{amount_to_add:.2f} X {item_name}")

In [ ]:
def add_new_item(item_name):
    """
    Asks amount, purchase and sell price of the item and adds it to the warehouse.
    Prints the operation performed on the screen.
    
    Arg:
    item_name (str): Product's name.
    """
    print("Quantità:")
    amount = is_float()
    print("Prezzo acquisto:")
    buy_price = is_float()
    print("Prezzo vendita:")
    selling_price = is_float()

    with open("warehouse.json") as json_file:
        data = json.load(json_file)
    new_item = {
        "amount": amount,
        "purchase price": buy_price,
        "sell price": selling_price
    }
    data[item_name] = new_item

    with open("warehouse.json", "w") as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)
    print(f"AGGIUNTO:{amount:.2f} X {item_name}")

In [ ]:
def is_float():
    """
    Checks that the value entered by the user is a float and is greater than zero.
    
    Return:
    amount (float): A valid value.
    """
    validity = False
    while validity == False:
        try:
            amount = float(input())
            while amount <= 0:
                print("Zero e valori negativi non sono ammessi. Riprova")
                amount = int(input())
            validity = True
        except:
            print("Il valore inserito non è un numero! Riprova")
    return amount

LIST PRODUCTS

In [ ]:
def show_items():
    """
    Print on-screen name/amount/price of products in the warehouse.
    """
    print("PRODOTTO QUANTITA' PREZZO")
    with open("warehouse.json") as json_file:
        data = json.load(json_file)
    for product, details in data.items():
        print(f"{product} {details['amount']:.2f} €{details['sell price']:.2f}")

SELL PRODUCTS

In [ ]:
def sell_item():
    """
    Records sales and keeps warehouse and balance up to date.
    """
    another_item = True
    items_list = []
    purchase_prices_list = []
    sell_prices_list = []
    while another_item == True:
        item_name = input("Nome del prodotto:")
        presence = check_item_presence(item_name)
        if presence == False:
            print("Questo prodotto non è presente in magazzino.")
        else:
            print("Quantità:")
            item_amount = is_float()
            price = read_price(item_name)
            available_amount = read_available_amount(item_name)
            availability = check_availability(item_amount, available_amount)
            while availability != True:
                answer = notify_and_ask(available_amount, item_name)
                if answer == "si":
                    print("Quantità:")
                    item_amount = is_float()
                    availability = check_availability(item_amount, available_amount)
                elif answer == "no":
                    break
            if availability == True:
                record_sell(item_name, item_amount, sell_prices_list, purchase_prices_list)
                add_to_items_list(item_name, item_amount, price, items_list)
            else:
                pass
        another_item = ask_for_another()
    total = print_sell(items_list, sell_prices_list)
    add_to_gross_profit(total)
    add_to_expenses(purchase_prices_list)
    check_if_zero()

In [ ]:
def read_price(item_name):
    """
    Recover the product's selling price.
    
    Arg:
    item_name (str): Product's name.
    
    Return:
    price (float): Product's sell price.
    """
    with open("warehouse.json") as json_file:
        data = json.load(json_file)
    price = data[item_name]["sell price"]
    return price

In [ ]:
def read_available_amount(item_name):
    """
    Recover the product's amount.
    
    Arg:
    item_name (str): Product's name.
    
    Return:
    available_amount (float): Product's available amount.
    """
    with open("warehouse.json") as json_file:
        data = json.load(json_file)
    available_amount = data[item_name]["amount"]
    return available_amount

In [ ]:
def check_availability(amount, available_amount):
    """
    Check that the required amount is in stock.
    
    Args:
    amount (float): amount entered by the user.
    available_amount (float): amount available in stock.
    
    Return:
    True (bool): amount available.
    False (bool) amount not available.
    """
    if amount > available_amount:
        return False
    else:
        return True

In [ ]:
def notify_and_ask(available, item_name):
    """
    Notifies that the requested quantity is not amount and asks if you want to try again.
    
    Arg:
    available (float): amount available in stock.
    item_name (str): Product's name.
    
    Return:
    answer (str): user answer to the question.
    """
    print("La quantità richiesta non è disponibile, "
        f"sono disponibili {available:.2f} unità di {item_name}")
    answer = input("Vuoi riprovare?(si/no):")
    while answer != "si" and answer != "no":
        print(f"{answer} non è una risposta valida, puoi digitare solo si o no. Riprova:")
        answer = input()
    return answer

In [ ]:
def record_sell(item_name, amount, sell_prices_list, purchase_prices_list):
    """
    Record a partial sale, save purchase price, sale price, and quantity in lists that will be useful later, 
    and update the inventory.
    
    Args:
    item_name (str): Product's name.
    amount (float): amount to sell.
    sell_prices_list (list): list containing the selling prices of products.
    purchase_prices_list (list): list containing the purchase prices of products.
    """
    with open("warehouse.json") as json_file:
        data = json.load(json_file)
    sell_price = data[item_name]["sell price"]
    purchase_price = data[item_name]["purchase price"]
    prev_amount = data[item_name]["amount"]
    data[item_name]["amount"] = prev_amount - amount
    total_sell_amount = amount * sell_price
    total_purchase_amount = amount * purchase_price
    sell_prices_list.append(total_sell_amount)
    purchase_prices_list.append(total_purchase_amount)

    with open("warehouse.json", "w") as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

In [ ]:
def add_to_items_list(item_name, amount, price, items_list):
    """
    Adds the product and its details to the shopping cart.
    
    Args:
    item_name (str): Product's name.
    amount (float): amount to sell.
    items_list (list): list containing tuples made by items names and their details.
    """
    new_tuple = (amount, item_name, price)
    items_list.append(new_tuple)

In [ ]:
def ask_for_another():
    """
    Asks the user if he want to add another product.
    
    Return:
    True (bool): user wants to add another product.
    False (bool): user don't want to add another product.
    """
    answer = input("Aggiungere un altro prodotto?(si/no):")
    while answer != "si" and answer != "no":
        print(f"{answer} non è una risposta valida, puoi digitare solo si o no. Riprova:")
        answer = input()
    if answer == "si":
        return True
    elif answer == "no":
        return False

In [ ]:
def print_sell(items_list, prices_list):
    """
    Print the total sale summary.
    
    Args:
    item_list (list): list containing tuples made by items names and their details.
    prices_list (list): list containing the selling prices of products.
    
    Return:
    total (float): Total amount to pay.
    """
    print("VENDITA REGISTRATA")
    for items in items_list:
        amount = items[0]
        item_name = items[1]
        price = items[2]
        print(f"{amount:.2f} X {item_name}: €{price:.2f}")
    total = calculate_total(prices_list)
    print(f"Totale: €{total:.2f}")
    return total

In [ ]:
def add_to_gross_profit(total):
    """
    Adds total to gross profit.
    
    Arg:
    total (float): Total amount payed by the customer.
    """
    with open("balance.json") as json_file:
        data = json.load(json_file)
    prev_gross_profit = data["gross profit"]
    data["gross profit"] = prev_gross_profit + total
    with open("balance.json", "w") as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

In [ ]:
def calculate_total(prices_list):
    """
    Sum all prices and calculate the total.
    
    Arg:
    prices_list (list): list containing the selling prices of products.
    
    Return:
    total (float): total prices sum.
    """
    total = 0
    for price in prices_list:
        total += price
    return total

In [ ]:
def add_to_expenses(purchase_prices_list):
    """
    Sum purchase prices to update the expenses balance.
    
    Arg:
    purchase_prices_list (list): list containing the purchase prices of products.
    """
    total = calculate_total(purchase_prices_list)
    with open("balance.json") as json_file:
        data = json.load(json_file)
    prev_expenses = data["expenses"]
    data["expenses"] = prev_expenses + total
    with open("balance.json", "w") as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

In [ ]:
def check_if_zero():
    """
    Checks for items with zero amount in stock and removes them.
    """
    with open("warehouse.json") as json_file:
        data = json.load(json_file)
    item_to_remove = []
    for item, details in data.items():
        if details["amount"] == 0:
            item_to_remove.append(item)
    for item in item_to_remove:
        del data[item]
    with open("warehouse.json", "w") as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)

SHOW PROFITS

In [ ]:
def show_profits():
    """
    Calculates gross and net profit and prints them on screen.
    """
    with open("balance.json") as json_file:
        data = json.load(json_file)
    gross_profit = data["gross profit"]
    expenses = data["expenses"]
    net_profit = gross_profit - expenses
    data["net profit"] = net_profit
    with open("balance.json", "w") as json_file:
        json.dump(data, json_file, indent=4, ensure_ascii=False)
    print(f"Profitto: lordo = €{gross_profit:.2f} netto = €{net_profit:.2f}")

COMMAND NOT VALID

In [ ]:
def wrong():
    """
    Alert the user that the entered command is invalid and shows the available commands on the screen.
    """
    print("Comando non valido.")
    show_commands()

MAIN

In [ ]:
import json

command = ""
while command != "chiudi":
    command = input("Inserisci un comando:")
    if command == "aiuto":
        show_commands()
    elif command == "aggiungi":
        add_to_warehouse()
    elif command == "elenca":
        show_items()
    elif command == "vendita":
        sell_item()
    elif command == "profitti":
        show_profits()
    elif command == "chiudi":
        print("Bye Bye")
        exit(0)
    else:
        wrong()